# Closer Look - Virginia Food Banks Dataset

In [38]:
# Imports
import pandas as pd
import folium
import json

In [39]:
df = pd.read_csv('OUTPUT/SUPPLY.csv')

# Convert relevant columns to numeric types
df['people_below_poverty_total'] = pd.to_numeric(df['people_below_poverty_total'], errors='coerce')
df['poverty_rate'] = df['poverty_rate'].str.rstrip('%').astype(float)
df['Pounds of Food Distributed'] = pd.to_numeric(df['Pounds of Food Distributed'], errors='coerce')
df['lbs_per_person_below_poverty'] = pd.to_numeric(df['lbs_per_person_below_poverty'], errors='coerce')

df_aggregated = df.groupby('Locality').agg({
    'people_below_poverty_total': 'mean',
    'poverty_rate': 'mean',
    'Pounds of Food Distributed': 'sum',
    'lbs_per_person_below_poverty': 'mean',
    'Supply': lambda x: x.mode()[0]  # Assuming mode is the most common supply level
}).reset_index()

df_aggregated = df_aggregated.round({
    'people_below_poverty_total': 2,
    'poverty_rate': 2,
    'Pounds of Food Distributed': 2,
    'lbs_per_person_below_poverty': 2
})

df_aggregated['poverty_rate'] = df_aggregated['poverty_rate'].astype(str) + '%'

In [40]:
va_map = folium.Map(location=[37.4316, -78.6569], zoom_start=8)

with open('va.geojson') as f:
    counties_geojson = json.load(f)

def get_supply_color(supply):
    if supply == '--':
        return 'red'
    elif supply == '-':
        return 'orange'
    elif supply == '○':
        return 'yellow'
    elif supply == '+':
        return 'lightgreen'
    elif supply == '++':
        return 'green'

for feature in counties_geojson['features']:
    locale_name = feature['properties']['name'] + " " + feature['properties']['TYPE']
    if locale_name in df['Locality'].values:
        supply_value = df[df['Locality'] == locale_name]['Supply'].values[0]
        feature['properties']['supply_color'] = get_supply_color(str(supply_value))
    else:
        print(locale_name)
        feature['properties']['supply_color'] = 'purple'

m = folium.Map(location=[37.5, -78.5], zoom_start=7)

folium.GeoJson(
    counties_geojson,
    style_function=lambda feature: {
        'fillColor': feature['properties']['supply_color'],
        'color': feature['properties']['supply_color'],
        'weight': 2,
        'fillOpacity': 0.3
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'supply_color'],
        aliases=['County:', 'Supply Level:']
    )
).add_to(m)

m.save('va_supply_map.html')


Winchester City
Williamsburg City
Waynesboro City
Virginia Beach City
Suffolk City
Staunton City
Salem City
Roanoke City
Richmond City
Radford City
Portsmouth City
Poquoson City
Petersburg City
Norton City
Norfolk City
Newport News City
Martinsville City
Manassas Park City
Manassas City
Lynchburg City
Lexington City
Hopewell City
Harrisonburg City
Hampton City
Galax City
Fredericksburg City
Franklin City
Falls Church City
Fairfax City
Emporia City
Danville City
Covington City
Colonial Heights City
Chesapeake City
Charlottesville City
Buena Vista City
Bristol City
Alexandria City
